# Importação e preparação dos dados

In [1]:
import pandas as pd

df = pd.read_csv('../dados/jurisprudencia_selecionada_excertos.CSV', sep = ';')
df.head()

,COD,NUM_ENUNCIADO,COD_AREA,DESCR_AREA,COD_TEMA,DESCR_TEMA,COD_SUBTEMA,DESCR_SUBTEMA,COD_DOC_TRAMITAVEL_ENUNCIADO,TEXTO_ENUNCIADO,ACORDAO,TIPO_PROCESSO
0,1400,1236,50,Responsabilidade,488,Solidariedade,261,Benefício previdenciário,54995437,Não comprovada a participação do beneficiário ...,Acórdão 297/2016 - PL,Tomada de Contas Especial
1,1700,1534,46,Finanças Públicas,981,Exportação,983,Petróleo,55025587,A operação ficta de exportação de plataformas ...,Acórdão 366/2016 - PL,Solicitação do Congresso Nacional
2,5700,5314,50,Responsabilidade,203,Multa,1021,Dosimetria,55455370,"No âmbito do TCU, a dosimetria da pena tem com...",Acórdão 944/2016 - PL,Acompanhamento
3,284,40,45,Direito Processual,162,Princípio da independência das instâncias,481,Decisão judicial,54773746,O princípio da independência das instâncias pe...,Acórdão 30/2016 - PL,Tomada de Contas Especial
4,298,54,49,Pessoal,141,Sistema S,142,Nepotismo,54773402,É vedado aos dirigentes das entidades do Siste...,Acórdão 55/2016 - PL,Representação


In [2]:
df.shape

(13312, 12)

In [3]:
df.groupby(['DESCR_AREA']).size()

DESCR_AREA
Competência do TCU          557
Contrato Administrativo     942
Convênio                    685
Desestatização              140
Direito Processual         1813
Finanças Públicas           328
Gestão Administrativa       339
Licitação                  2765
Pessoal                    3396
Responsabilidade           2347
dtype: int64

In [4]:
areas = df.groupby(['DESCR_AREA']).groups.keys()
areas

dict_keys(['Competência do TCU', 'Contrato Administrativo', 'Convênio', 'Desestatização', 'Direito Processual', 'Finanças Públicas', 'Gestão Administrativa', 'Licitação', 'Pessoal', 'Responsabilidade'])

In [5]:
from sklearn.preprocessing import LabelBinarizer

lbArea = LabelBinarizer()
lbArea.fit([x for x in areas])
lbArea.classes_

array(['Competência do TCU', 'Contrato Administrativo', 'Convênio',
       'Desestatização', 'Direito Processual', 'Finanças Públicas',
       'Gestão Administrativa', 'Licitação', 'Pessoal',
       'Responsabilidade'], dtype='<U23')

In [6]:
y = lbArea.transform(df['DESCR_AREA'])
y.shape

(13312, 10)

In [7]:
from keras.preprocessing.text import Tokenizer
import numpy as np

limite_texto = 200
dim_vetor = 100

tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['TEXTO_EXCERTO'])

word_index = tokenizer.word_index
vocabulario = len(word_index) + 1
print('Found %s unique tokens.' % len(word_index))

sequences = tokenizer.texts_to_sequences(df['TEXTO_EXCERTO'])

Using TensorFlow backend.


Found 15387 unique tokens.


In [8]:
from keras.preprocessing.sequence import pad_sequences

x = pad_sequences(sequences, maxlen=limite_texto)

print('Shape of data tensor:', x.shape)

Shape of data tensor: (13312, 200)


In [9]:
from gensim.models import Word2Vec

model = Word2Vec.load('../vocabularios/modelo-acordaos.w2v')

/home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [10]:
# create a weight matrix for words in training docs

embedding_matrix = np.zeros((vocabulario, 100))

ok = 0
for word, i in tokenizer.word_index.items():
    if word in model.wv:
        embedding_matrix[i] = model.wv[word]
        ok += 1
print('Vocabulario:', i)
print('Encontrados no modelo:', ok, '=', ok * 100. / i)

Vocabulario: 15387
Encontrados no modelo: 14572 = 94.70332098524729


In [11]:
nok = 0
for word, i in tokenizer.word_index.items():
    if word not in model.wv:
        print(word)
        nok += 1
        if nok > 20: break

fazê
dsic
in01
“quintos”
'quintos'
rogação
'opção'
“opção”
“a”
“b”
po5
“s”
utilizá
po7
executá
's'
'a'
ds5
“jogo
‘quintos’
“ou


# Treinamento

## Embedding com pesos fixos

In [12]:
from keras.layers import Embedding

embedding = Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=False)

W0912 11:10:41.063221 139684724791104 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



In [13]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding
from keras.layers.core import Dropout

model = Sequential()
model.add(embedding)
model.add(Flatten())
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.6))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.6))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta',  metrics=["categorical_accuracy"])
model.summary()

W0912 11:10:47.102676 139684724791104 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0912 11:10:47.180051 139684724791104 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0912 11:10:47.202162 139684724791104 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0912 11:10:47.203325 139684724791104 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecate

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 200, 100)          1538800   
_________________________________________________________________
flatten_1 (Flatten)          (None, 20000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 2048)              40962048  
_________________________________________________________________
dropout_1 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                10250     
Total para

In [14]:
history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

W0912 11:11:14.341386 139684724791104 deprecation.py:323] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 20s 2ms/step - loss: 1.9043 - categorical_accuracy: 0.5202 - val_loss: 1.1887 - val_categorical_accuracy: 0.6057
Epoch 2/20
10649/10649 [==============================] - 17s 2ms/step - loss: 1.0379 - categorical_accuracy: 0.6851 - val_loss: 1.0639 - val_categorical_accuracy: 0.6662
Epoch 3/20
10649/10649 [==============================] - 17s 2ms/step - loss: 0.7847 - categorical_accuracy: 0.7590 - val_loss: 0.9648 - val_categorical_accuracy: 0.6947
Epoch 4/20
10649/10649 [==============================] - 17s 2ms/step - loss: 0.6178 - categorical_accuracy: 0.8131 - val_loss: 1.1071 - val_categorical_accuracy: 0.6767
Epoch 5/20
10649/10649 [==============================] - 17s 2ms/step - loss: 0.4717 - categorical_accuracy: 0.8557 - val_loss: 1.0765 - val_categorical_accuracy: 0.6688
Epoch 6/20
10649/10649 [==============================] - 17s 2ms/step - loss: 0.3764 - categori

In [15]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=False))
model.add(Flatten())
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta',  metrics=["categorical_accuracy"])

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 18s 2ms/step - loss: 1.7098 - categorical_accuracy: 0.5764 - val_loss: 1.1012 - val_categorical_accuracy: 0.6384
Epoch 2/20
10649/10649 [==============================] - 17s 2ms/step - loss: 0.7368 - categorical_accuracy: 0.7671 - val_loss: 1.1673 - val_categorical_accuracy: 0.6207
Epoch 3/20
10649/10649 [==============================] - 17s 2ms/step - loss: 0.4198 - categorical_accuracy: 0.8613 - val_loss: 1.3048 - val_categorical_accuracy: 0.6395
Epoch 4/20
10649/10649 [==============================] - 17s 2ms/step - loss: 0.2341 - categorical_accuracy: 0.9238 - val_loss: 1.5454 - val_categorical_accuracy: 0.6699
Epoch 5/20
10649/10649 [==============================] - 17s 2ms/step - loss: 0.1496 - categorical_accuracy: 0.9528 - val_loss: 1.7734 - val_categorical_accuracy: 0.6902
Epoch 6/20
10649/10649 [==============================] - 17s 2ms/step - loss: 0.1072 - categori

In [16]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=False))
model.add(Flatten())
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta',  metrics=["categorical_accuracy"])

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 17s 2ms/step - loss: 1.4884 - categorical_accuracy: 0.5674 - val_loss: 1.2033 - val_categorical_accuracy: 0.5997
Epoch 2/20
10649/10649 [==============================] - 16s 2ms/step - loss: 0.8624 - categorical_accuracy: 0.7149 - val_loss: 1.1532 - val_categorical_accuracy: 0.6489
Epoch 3/20
10649/10649 [==============================] - 16s 2ms/step - loss: 0.6100 - categorical_accuracy: 0.7977 - val_loss: 1.1342 - val_categorical_accuracy: 0.6786
Epoch 4/20
10649/10649 [==============================] - 16s 1ms/step - loss: 0.4055 - categorical_accuracy: 0.8658 - val_loss: 1.3536 - val_categorical_accuracy: 0.6812
Epoch 5/20
10649/10649 [==============================] - 16s 1ms/step - loss: 0.2538 - categorical_accuracy: 0.9144 - val_loss: 1.4928 - val_categorical_accuracy: 0.6695
Epoch 6/20
10649/10649 [==============================] - 16s 2ms/step - loss: 0.1639 - categori

## Embedding com pesos ajustáveis

In [17]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding
from keras.layers.core import Dropout

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(Flatten())
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.6))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.6))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta',  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

W0912 11:27:55.429187 139684724791104 nn_ops.py:4224] Large dropout rate: 0.6 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
W0912 11:27:55.453449 139684724791104 nn_ops.py:4224] Large dropout rate: 0.6 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 200, 100)          1538800   
_________________________________________________________________
flatten_4 (Flatten)          (None, 20000)             0         
_________________________________________________________________
dense_10 (Dense)             (None, 2048)              40962048  
_________________________________________________________________
dropout_7 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_11 (Dense)             (None, 1024)              2098176   
_________________________________________________________________
dropout_8 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_12 (Dense)             (None, 10)                10250     
Total para

In [18]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(Flatten())
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta',  metrics=["categorical_accuracy"])

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 19s 2ms/step - loss: 1.4557 - categorical_accuracy: 0.5836 - val_loss: 1.2753 - val_categorical_accuracy: 0.5546
Epoch 2/20
10649/10649 [==============================] - 17s 2ms/step - loss: 0.7051 - categorical_accuracy: 0.7716 - val_loss: 1.1230 - val_categorical_accuracy: 0.6440
Epoch 3/20
10649/10649 [==============================] - 17s 2ms/step - loss: 0.3966 - categorical_accuracy: 0.8683 - val_loss: 1.4748 - val_categorical_accuracy: 0.6538
Epoch 4/20
10649/10649 [==============================] - 17s 2ms/step - loss: 0.2005 - categorical_accuracy: 0.9361 - val_loss: 1.5702 - val_categorical_accuracy: 0.6928
Epoch 5/20
10649/10649 [==============================] - 17s 2ms/step - loss: 0.1198 - categorical_accuracy: 0.9618 - val_loss: 1.7197 - val_categorical_accuracy: 0.6891
Epoch 6/20
10649/10649 [==============================] - 17s 2ms/step - loss: 0.0857 - categori

In [19]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(Flatten())
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta',  metrics=["categorical_accuracy"])

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 19s 2ms/step - loss: 1.5143 - categorical_accuracy: 0.5591 - val_loss: 1.2118 - val_categorical_accuracy: 0.5790
Epoch 2/20
10649/10649 [==============================] - 18s 2ms/step - loss: 0.8747 - categorical_accuracy: 0.7174 - val_loss: 0.9873 - val_categorical_accuracy: 0.6774
Epoch 3/20
10649/10649 [==============================] - 17s 2ms/step - loss: 0.5976 - categorical_accuracy: 0.8064 - val_loss: 1.0315 - val_categorical_accuracy: 0.6910
Epoch 4/20
10649/10649 [==============================] - 17s 2ms/step - loss: 0.3818 - categorical_accuracy: 0.8750 - val_loss: 1.3448 - val_categorical_accuracy: 0.6891
Epoch 5/20
10649/10649 [==============================] - 18s 2ms/step - loss: 0.2392 - categorical_accuracy: 0.9204 - val_loss: 1.4156 - val_categorical_accuracy: 0.7067
Epoch 6/20
10649/10649 [==============================] - 17s 2ms/step - loss: 0.1607 - categori